In [15]:
#add versions to libraries
import pandas as pd # data analysis and manipulation tool
import numpy as np # Numerical computing tools
import seaborn as sns  # visualization library
import matplotlib.pyplot as plt  # another visualization library
import scipy as sp

import re
import typing

In [16]:
df = pd.read_csv('virus_hw2.csv')

In [17]:
soc = [ item for item in df.AvgTimeOnStuding if item>0]
len(soc)

750

# Change TestResultsCode to three labels, disease, contagious, risk

In [18]:
#test result classified by labels
li = df.TestResultsCode.tolist()
labels =  [ item.split('_') for item in li ] 
for item in labels:
    if len(item)==4:
        add =  item[0]+item[1]     
        item = item.insert( 0, add  )
for item in labels:
    if 'not' in item:
        item.remove('not')
    if 'detected' in item:
        item.remove('detected')


In [19]:
#one-hot encode the test results
disease = [ la[0] for la in labels ]
spread = [  la[1] for la in labels  ]
risk = [  la[2] for la in labels  ]

disease_encode = pd.Series( disease  ).str.get_dummies()
spread_encode = pd.Series( spread  ).str.get_dummies()
risk_encode = pd.Series( risk  ).str.get_dummies()

disease_encode = pd.DataFrame( disease_encode )
spread_encode = pd.DataFrame( spread_encode )
risk_encode = pd.DataFrame( risk_encode)


In [20]:
#one-hot encode Sex column
#sex_encode = pd.Series( df.loc[:,'Sex'] ).str.get_dummies()
#sex_encode = pd.DataFrame( sex_encode ).drop('F',axis=1) #classify male/not male
#sex_encode = sex_encode.rename(columns={"M": "Sex"})
#
#df = df.drop( 'Sex' , axis=1)
#df = pd.concat( [df, sex_encode], axis=1 )


In [21]:
#interate one hot encoding of test results back to df
df=df.drop(['PatientID', 'Address', 'CurrentLocation'],axis=1)
df2 = df
df2 = df2.drop(columns = 'TestResultsCode')


In [22]:
results = pd.concat( [risk_encode, spread_encode, disease_encode], axis=1 )
results = results.drop(['NotSpreader', 'NotatRisk'], axis=1)


# test, train split

In [23]:
from sklearn.model_selection import train_test_split

X_train,  X_val, y_train, y_val = train_test_split( df2, results, test_size=0.33, random_state=33 ) #tr is test results numerically coded
X_val, X_test, y_val, y_test = train_test_split( X_val, y_val , test_size=0.4, random_state=33)

# Data Imputation Linear Regression

In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

#one-hot encode Sex column
sex_encode = pd.Series( X_train[X_train['Sex'].notnull()].loc[:,'Sex'] ).str.get_dummies()
sex_encode = pd.DataFrame( sex_encode ).drop('F',axis=1) #classify male/not male

c = ['AvgMinSportsPerDay', 'DisciplineScore', 'HappinessScore']
X = X_train[X_train['Sex'].notnull()].loc[:, c]
X = X.fillna(X.mean()) # fill non-Sex  columns with mean value
X = X.values
y = sex_encode.values


np.random.seed(42)
kf = KFold(n_splits=4)
scores = []
for train_index, test_index in kf.split(X):
    X_train_, X_test_ = X[train_index], X[test_index]
    y_train_, y_test_ = y[train_index], y[test_index]
    
    clf = LinearRegression()
    clf.fit(X_train_, y_train_)
    y_test_pred = clf.predict(X_test_)
    
    scores.append(r2_score(y_test_, y_test_pred))



# fill nan values in Sex column
## The  R2  score measures how much better than baseline linear regression performs, where baseline is flat regression against the mean. In this case that baseline performance (an  R2  of 0) is the performance of replacing the missing values with the mean of the observed values.

In [25]:
#one-hot encode Sex column
sex_encode = pd.Series( X_train[X_train['Sex'].notnull()].loc[:,'Sex'] ).str.get_dummies()
sex_encode = pd.DataFrame( sex_encode ).drop('F',axis=1) #classify male/not male
y_train_ = sex_encode.values


#predict nan values in Sex Column

c = ['AvgMinSportsPerDay', 'DisciplineScore', 'HappinessScore']
X_train_mod = X_train.loc[:, c]
X_train_mod = X_train_mod.fillna(X_train_mod.mean())
X_train_mod_fit = X_train_mod[X_train['Sex'].notnull()] # x_train values of c columns where sex is not null

X_train_mod_pred = X_train_mod[X_train['Sex'].isnull()].loc[:, c]

clf = LinearRegression()
clf.fit(X_train_mod_fit, y_train_) #train model on non -Nan values#y_test_pred = clf.predict(X_pred) #predict on Nan values
pred = clf.predict(X_train_mod_pred)


# Sequential Forward Selector

In [14]:
#REMOVED LOCATION FROM FEATURES

cols =['AgeGroup','AvgHouseholdExpenseOnPresents','AvgHouseholdExpenseOnSocialGames',
        'AvgHouseholdExpenseParkingTicketsPerYear','AvgMinSportsPerDay','AvgTimeOnSocialMedia','AvgTimeOnStuding','BMI',
        'DisciplineScore','HappinessScore','Job','NrCousins','StepsPerYear','SyndromeClass','TimeOnSocialActivities']

X_train_sfs = X_train[cols]
X_train_sfs = X_train_sfs.fillna(X_train_sfs.mean())

from sklearn.neighbors import KNeighborsClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS #import from mlxtend library
knn = KNeighborsClassifier(n_neighbors=2) # ml_algo used = knn
sfs = SFS(knn, 
           k_features=10,  
           forward=True, # if forward = True then SFS otherwise SBS
           floating=False, 
           verbose=2,
           scoring='accuracy'
           )


#after applying sfs fit the data:
sfs.fit(X_train_sfs, y_train)
sfs.k_feature_names_  



ModuleNotFoundError: No module named 'mlxtend'